In [1]:
import io
import os
import sys
import numpy as np
import tensorflow as tf
from PIL import Image
from flask import Flask, jsonify, request, render_template, Response, flash, redirect, url_for, session
import base64
import cv2
from datetime import datetime
from flask_mysqldb import MySQL
import MySQLdb.cursors
from flask_login import LoginManager, login_user, login_required, logout_user, current_user

In [2]:
app = Flask(__name__, template_folder="./Website")
app.secret_key = "jun"

# initialize login_manager
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

app.config["MYSQL_HOST"] = "localhost"
app.config["MYSQL_USER"] = "root"
app.config["MYSQL_PASSWORD"] = ""
app.config["MYSQL_DB"] = "tics"
mysql = MySQL(app)

class User:
    def __init__(self, StudentID, StudentName, ClassID, StudentParentName, StudentParentNo, active=True):
        self.StudentID = StudentID
        self.StudentName = StudentName
        self.ClassID = ClassID
        self.StudentParentName = StudentParentName
        self.StudentParentNo = StudentParentNo
        self.active = active
        self.is_authenticated = True
        
    def is_active(self):
        return self.active

    def get_id(self):
        return self.StudentID
    
    def getStudentName(self):
        return self.StudentName

@app.route("/")
def home():
    return render_template("Home.html", user=current_user)

@app.route("/register", methods=["GET", "POST"])
def register():
    if request.method == "POST" and "StudentName" in request.form:
        StudentName = request.form.get("StudentName")
        StudentParentName = request.form.get("StudentParentName")
        StudentParentNo = request.form.get("StudentParentNo")
        ClassID = request.form.get("ClassID")
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute("SELECT * FROM students WHERE StudentName=%s", (StudentName,))
        user = cursor.fetchone()
        if user:
            flash("User already exists. Please log in!", "danger")
            return redirect(url_for("login"))
        else:
            cursor.execute("INSERT INTO students(StudentName, StudentParentName, StudentParentNo, ClassID) VALUES(%s, %s, %s, %s)", (StudentName, StudentParentName, StudentParentNo, ClassID,))
            mysql.connection.commit()
            flash("User registered successfully. Please log in!", "success")
            return redirect(url_for("login"))
    return render_template("Register.html", user=current_user)

@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST" and "StudentName" in request.form:
        StudentName = request.form.get("StudentName")
        ClassID = request.form.get("ClassID")
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute("SELECT * FROM students WHERE StudentName=%s AND ClassID=%s", (StudentName, ClassID,))
        user = cursor.fetchone()
        if user:
            user = User(user["StudentID"], user["StudentName"], user["ClassID"], user["StudentParentName"], user["StudentParentNo"])
            login_user(user, remember = True)  
            flash("Logged in successfully!", "success")
            return redirect(url_for("userhome"))
        else:
            flash("Login Unsuccessful. Please check your credentials!", "danger")
    return render_template("login.html", user=current_user)

@app.route("/logout")
@login_required
def logout():
    logout_user()
    flash("You have logged out successfully!", "success")
    return redirect(url_for("home"))

@login_manager.user_loader
def load_user(userid):
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute("SELECT * FROM students WHERE StudentID=%s", (userid,))
    student = cursor.fetchone()
    if student:
        return User(student["StudentID"], student["StudentName"], student["ClassID"], student["StudentParentName"], student["StudentParentNo"])
    return None

@app.route("/userhome")
@login_required
def userhome():
    print(current_user.getStudentName())
    return render_template("User.html", user=current_user)

@app.route("/game")
@login_required
def game():
    return render_template("Game.html", user=current_user)

@app.route("/about")
def about():
    return render_template("About.html", user=current_user)

@app.route("/draw")
@login_required
def draw():
    return render_template("Draw.html", user=current_user)

def gen_frames(current_user):
    video_capture = cv2.VideoCapture(0)
    labels_dict = {0: 'No Tic', 1: 'Tic'}
    color_dict = {0: (0, 0, 255), 1: (0, 255, 0)}
    img_size = 80
    face_cascade = cv2.CascadeClassifier(r'./haarcascade_frontalface_default.xml')
    model = tf.keras.models.load_model("./model/TICs.h5")
    tic_detected = False
    tic_start_time = None
    
    while True:
        success, frame = video_capture.read()
        if not success:
            break
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = face_cascade.detectMultiScale(frame, 1.3, 5)

            for x, y, w, h in faces:
                face_img = frame[y:y + w, x:x + h]
                resized = cv2.resize(face_img, (img_size, img_size))
                normalized = cv2.normalize(resized, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                reshaped = np.reshape(normalized, (1, img_size, img_size, 3))
                result = model.predict(reshaped)
                label = np.argmax(result, axis=1)[0]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color_dict[label], 2)
                cv2.putText(frame, labels_dict[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                
                if label == 1:
                    if not tic_detected:
                        tic_detected = True
                        tic_start_time = datetime.now()
                        
                    elif (datetime.now() - tic_start_time).total_seconds() > 3:
                        print("Timestamp: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                        print(current_user.getStudentName())
                        tic_detected = False
                        tic_start_time = None
                else:
                    tic_detected = False
                    tic_start_time = None
                    
            resized_img = cv2.resize(frame, (1000, 700))
            ret, buffer = cv2.imencode('.jpg', resized_img)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
     
@app.route("/camera", methods=["GET", "POST"])
@login_required
def camera():
    if request.method == "POST":
        if request.form["submit_button"] == "Start":
            flash("Camera has started!", "success")
            return redirect(url_for("classify", user=current_user))
        else:
            return redirect(url_for("camera"))
    return render_template("Camera.html", user=current_user)
        
@app.route("/classify", methods=["GET"])
def classify():
    try:
        return Response(gen_frames(current_user), mimetype="multipart/x-mixed-replace; boundary=frame")
    except:
        return "Error: Failed to generate video frames."

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    app.debug= True    
    run_simple("localhost", 5001, app)

 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [12/Feb/2023 19:21:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Feb/2023 19:21:53] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Feb/2023 19:21:53] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Feb/2023 19:22:00] "GET /camera HTTP/1.1" 200 -
127.0.0.1 - - [12/Feb/2023 19:22:00] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Feb/2023 19:22:00] "GET /static/Index.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Feb/2023 19:22:14] "GET /classify HTTP/1.1" 200 -
Error on request:
Traceback (most recent call last):
  File "c:\Users\lowyi\anaconda3\envs\asp\lib\site-packages\werkzeug\serving.py", line 335, in run_wsgi
    execute(self.server.app)
  File "c:\Users\lowyi\anaconda3\envs\asp\lib\site-packages\werkzeug\serving.py", line 324, in execute
    for data in application_iter:
  File "c:\Users\lowyi\anaconda3\envs\asp\lib\site-packages\werkzeug\wsgi.py", line 500, in __next__
    return self._next()

Timestamp:  2023-02-12 19:23:16


127.0.0.1 - - [12/Feb/2023 19:23:47] "GET /userhome HTTP/1.1" 200 -
127.0.0.1 - - [12/Feb/2023 19:23:47] "GET /static/index.css HTTP/1.1" 304 -


jun


127.0.0.1 - - [12/Feb/2023 19:23:47] "GET /static/Index.js HTTP/1.1" 304 -
